In [9]:
import os
import os.path

import numpy as np
import pandas as pd

# Data Challenge

Files are stored in an S3 bucket. The purpose here is to fully analyze the data and make some predictions.

This workbook was exported to a Python script and the resulting code was checked for PEP8 problems. Any problems were corrected.

## Bucket Contents and File Download

In [2]:
s3 = boto3.resource('s3')
bucket_name = "postie-testing-assets"

In [3]:
test = s3.Bucket(bucket_name)

In [4]:
s3.meta.client.head_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'application/xml',
   'date': 'Mon, 16 Oct 2017 18:06:53 GMT',
   'server': 'AmazonS3',
   'transfer-encoding': 'chunked',
   'x-amz-bucket-region': 'us-east-1',
   'x-amz-id-2': 'YhUEo61GDGSwz1qOpFGJl+C9Sxal34XKRYzOI0TF49PsSSGsbGg2Y6xwbf07z+KHIKusPIYkjxE=',
   'x-amz-request-id': 'DDD0C4B61BDF320E'},
  'HTTPStatusCode': 200,
  'HostId': 'YhUEo61GDGSwz1qOpFGJl+C9Sxal34XKRYzOI0TF49PsSSGsbGg2Y6xwbf07z+KHIKusPIYkjxE=',
  'RequestId': 'DDD0C4B61BDF320E',
  'RetryAttempts': 0}}

In [72]:
for key in test.objects.all():
    print(key.key)

2017-07-01.csv
2017-07-02.csv
2017-07-03.csv


An alternative is to download the data using

```aws s3 cp --recursive s3://my_bucket_name local_folder```


## Data Issues

1. Column names have spaces, so we need to remove them. A more sophisticated method would do this on import. However, processing the first row in this way may slow down the import process, particularly if the files are much larger. There are ways to read chunks via ```read_csv```, which can be used in a class to get the first line of the file, process it as a header, then continue reading the rest of the file in chunks. This is probably the best way to read many large files.
2. Placeholder is blank (NaN) for two files. But is this needed?
3. The file labeled "2017-07-01" has transactions for 7/1/2017 and 7/2/2017.
4. The file labeled "2017-07-02" has transactions for 7/2/2017 and 7/3/2017.
5. The file labeled "2017-07-03" has transactions only for 7/3/2017.
6. The url field is not url-encoded, so it's easy to use split to get various pieces.
7. Unless we know otherwise, "Ylang-ylang" is the name of an item, as opposed to "Ylang&ylang", which is *two* different items. This does affect item counts.
8. There are two website id's, but one website id has two separate domain names: store.example.com, and www.example.com. This affects counts and also reporting if using the domain name. **Handling the domain names is very dependent on this dataset - no effort was made to write a more general solution**.

## Observations

1. Analyst is correct - July 3rd *file* has sales of $164,065. There are fewer sales and customers on the 3rd than the other two days, so the sales amount is lower. However, there are problems with the data files, so there are sales recorded on the 3rd found in files labeled for the 2nd and 3rd. There are sales recorded on the 2nd found in the files labeled for the 1st and 2nd. To get accurate sales data we will have to group by transaction date - the sales date parsed from the *timestamp* field.
2. The *placeholder* field is empty for two files, so it's probably not meaningful.
3. *app_version* may or may not be meaningful. There is only one app_version for the files labeled "2017-07-01" and "2017-07-02". The remaining file has two app_versions - 1.1 and 1.2. This field is probably not meaningful for prediction, but sales on the 3rd should be examined for the different app_versions.

## Plans
- Get the items. But should we make new columns in the original dataset?
- Items are paired with item counts.
- We would like to know the sales by domain and by item.
- We would like to know the corrected sales by date.
- Average sales is not meaningful - explain why.
- Sales prediction at the aggregate level is not meaningful - there are only two values per domain.
- So do we predict sales at the item level? If so, how?
- What do we do with the *app_version*?
- What are the values in *placeholder*?
- Explain why average sales for a day is not meaningful.

In [99]:
fr = pd.DataFrame()
col_names = ["timestamp", "website_id", "customer_id", "app_version", "placeholder", "checkout_amount", "url"]
data_report = []
for fname in os.listdir("data"):
    ffr = pd.read_csv(os.path.join("data", fname),
                      header=0, names=col_names,
                      infer_datetime_format=True, parse_dates=[0])
    transaction_date = fname.split(".")[0]
    ffr["transaction_date"] = transaction_date
    file_date = ffr.timestamp.apply(lambda x: x.strftime('%Y-%m-%d'))
    domain_items = ffr.url.str.split("//")  # assumption is we always have a "valid" url - "http://" is not found
    domain_name = domain_items.apply(lambda x: x[1].split("/")[0])
    ffr["domain_name"] = domain_name.apply(lambda x: x if not "example.com" in x else ".".join(x.split(".")[1:]))
    item_names = domain_items.apply(lambda x: x[1].split("?")[1])  # see comments above
    print("date {} has {} sales for rows {} and unique dates {}".format(fname, ffr.checkout_amount.sum(),
                                                                        ffr.shape[0], file_date.unique().shape[0]))
    data_report.append({"transaction_date": transaction_date, "sales": ffr.checkout_amount.sum(),
                        "placeholder_nan": sum(ffr.placeholder.isnull()),
                        "n_rows": ffr.shape[0],
                        "n_websites": ffr.website_id.unique().shape[0],
                        "n_customers": ffr.customer_id.unique().shape[0],
                        "n_app_versions": ffr.app_version.unique().shape[0],
                        "n_dates": file_date.unique().shape[0]})
    fr = fr.append(ffr)

date 2017-07-02.csv has 183294.0 sales for rows 11573 and unique dates 2
date 2017-07-01.csv has 241491.0 sales for rows 11634 and unique dates 2
date 2017-07-03.csv has 164065.0 sales for rows 9981 and unique dates 1


In [103]:
pd.DataFrame(data_report)

,n_app_versions,n_customers,n_dates,n_rows,n_websites,placeholder_nan,sales,transaction_date
0,1,8198,2,11573,2,11573,183294.0,2017-07-02
1,1,8226,2,11634,2,11634,241491.0,2017-07-01
2,2,7412,1,9981,2,1792,164065.0,2017-07-03


In [75]:
fr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33188 entries, 0 to 9980
Data columns (total 9 columns):
timestamp           33188 non-null datetime64[ns]
website_id          33188 non-null int64
customer_id         33188 non-null int64
app_version         33188 non-null float64
placeholder         8189 non-null object
checkout_amount     33188 non-null float64
url                 33188 non-null object
transaction_date    33188 non-null object
domain_name         33188 non-null object
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 2.5+ MB


In [76]:
fr.head()

,timestamp,website_id,customer_id,app_version,placeholder,checkout_amount,url,transaction_date,domain_name
0,2017-07-02 07:00:35,123,9418,1.1,NaN,6.0,http://www.example.com/store/?Ume=1,2017-07-02,www.example.com
1,2017-07-02 07:00:37,124,3872,1.1,NaN,3.0,http://xyz.com/checkout?Prairie+Potato=1,2017-07-02,xyz.com
2,2017-07-02 07:00:47,123,3090,1.1,NaN,4.0,http://www.example.com/store/?Hazelnut=1,2017-07-02,www.example.com
3,2017-07-02 00:00:51,124,9556,1.1,NaN,6.0,http://xyz.com/checkout?Ume=1,2017-07-02,xyz.com
4,2017-07-02 00:01:02,124,8845,1.1,NaN,6.0,http://xyz.com/checkout?Bignay=1,2017-07-02,xyz.com


In [93]:
pd.pivot_table(fr, values="checkout_amount", index="transaction_date", columns="domain_name",
               aggfunc=[np.mean, np.median, np.sum])

mean                 median                 sum  \
domain_name      example.com    xyz.com example.com xyz.com example.com   
transaction_date                                                          
2017-07-01         31.609096  10.112549         6.0     6.0    182100.0   
2017-07-02         21.863808   9.721052         6.0     6.0    127466.0   
2017-07-03         11.214861  21.609571         6.0     6.0     55693.0   

                            
domain_name        xyz.com  
transaction_date            
2017-07-01         59391.0  
2017-07-02         55828.0  
2017-07-03        108372.0

In [49]:
ffr.url.unique()

array(['http://xyz.com/checkout?Ylang-ylang=1',
       'http://xyz.com/checkout?Prairie+Potato=1',
       'http://www.example.com/store/?Round+Kumquat=1',
       'http://www.example.com/store/?Bignay=1',
       'http://www.example.com/store/?European+Grape=1',
       'http://www.example.com/store/?Hazelnut=1',
       'http://www.example.com/store/?Hazelnut=1&Ylang-ylang=2&Round+Kumquat=1&Mabolo=2&European+Grape=1&Natal+Orange=1&Bignay=2',
       'http://www.example.com/store/?Prairie+Potato=1',
       'http://xyz.com/checkout?Ume=1',
       'http://www.example.com/store/?Ume=1',
       'http://www.example.com/store/?Natal+Orange=1',
       'http://xyz.com/checkout?Hazelnut=1',
       'http://www.example.com/store/?Mabolo=1',
       'http://www.example.com/store/?Ylang-ylang=1',
       'http://xyz.com/checkout?Bignay=1',
       'http://xyz.com/checkout?Round+Kumquat=1',
       'http://xyz.com/checkout?Black%2FWhite+Pepper=1',
       'http://www.example.com/store/?Hazelnut=2&Ume=2&Prairi

In [70]:
ffr.url.str.split("//").apply(lambda x: x[1].split("?")[1])

0                                           Ylang-ylang=1
1                                        Prairie+Potato=1
2                                         Round+Kumquat=1
3                                                Bignay=1
4                                         Round+Kumquat=1
5                                        European+Grape=1
6                                        European+Grape=1
7                                        Prairie+Potato=1
8                                              Hazelnut=1
9       Hazelnut=1&Ylang-ylang=2&Round+Kumquat=1&Mabol...
10                                       Prairie+Potato=1
11                                                  Ume=1
12                                                  Ume=1
13                                         Natal+Orange=1
14                                             Hazelnut=1
15                                                  Ume=1
16                                       Prairie+Potato=1
17            

In [81]:
fr.app_version.unique()

array([ 1.1,  1.2])